In [ ]:
import numpy as np
import random

# Definimos las capacidades de generación de cada planta (en GW)
capacidad_plantas = np.array([3, 6, 5, 4])

# Definimos la demanda energética de cada ciudad (en GW)
demanda_ciudades = np.array([4, 3, 5, 3])

# Costos de transporte de energía por GW entre cada planta y cada ciudad
costo_transporte = np.array([[1, 4, 3, 6],
                              [4, 1, 4, 5],
                              [3, 4, 1, 4],
                              [6, 5, 4, 1]])

# Costos de generación por cada GW-H producido en cada planta
costo_generacion = np.array([680, 720, 660, 750])

def calcular_costo(solucion):
    """
    Calcula el costo total de una distribución de energía.
    Tiene en cuenta tanto el costo de transporte como el costo de generación.
    """
    return np.sum(solucion * (costo_transporte + costo_generacion[:, None] * 1e3))

def generar_solucion():
    """
    Crea una distribución inicial de energía asegurando que la demanda de cada ciudad sea satisfecha
    y que ninguna planta supere su capacidad.
    """
    solucion = np.zeros((4, 4))
    capacidad_restante = capacidad_plantas.copy()
    demanda_restante = demanda_ciudades.copy()

    # Asignamos energía priorizando el menor costo de transporte
    for j in range(4):  # Para cada ciudad
        plantas_disponibles = np.argsort(costo_transporte[:, j])  # Ordenar plantas por costo de transporte
        for i in plantas_disponibles:
            if demanda_restante[j] > 0 and capacidad_restante[i] > 0:
                asignacion = min(demanda_restante[j], capacidad_restante[i])
                solucion[i, j] = asignacion
                demanda_restante[j] -= asignacion
                capacidad_restante[i] -= asignacion

    return solucion

def cruzar(padre1, padre2):
    """
    Mezcla dos soluciones (padres) para crear una nueva solución (hijo).
    """
    hijo = np.copy(padre1)
    punto = random.randint(1, 3)  # Elegimos un punto de corte aleatorio
    hijo[:punto, :] = padre2[:punto, :]
    return hijo

def mutar(solucion):
    """
    Introduce cambios en una solución para explorar nuevas combinaciones.
    Se selecciona un punto aleatorio y se transfiere parte de la energía asignada.
    """
    i, j = random.randint(0, 3), random.randint(0, 3)
    i2 = random.randint(0, 3)

    if solucion[i, j] > 0 and capacidad_plantas[i2] - np.sum(solucion[i2]) > 0:
        transferencia = min(solucion[i, j], capacidad_plantas[i2] - np.sum(solucion[i2]))
        solucion[i, j] -= transferencia
        solucion[i2, j] += transferencia
    return solucion

def es_solucion_valida(solucion):
    """
    Verifica si una solución es válida.
    - Cada ciudad debe recibir exactamente su demanda de energía.
    - Ninguna planta puede exceder su capacidad.
    """
    return np.allclose(np.sum(solucion, axis=0), demanda_ciudades) and np.all(np.sum(solucion, axis=1) <= capacidad_plantas)

def algoritmo_genetico(tamano_poblacion=50, generaciones=1000):
    """
    Ejecuta un algoritmo genético para encontrar la mejor distribución de energía,
    minimizando costos de transporte y generación.
    """
    poblacion = [generar_solucion() for _ in range(tamano_poblacion)]

    for gen in range(generaciones):
        poblacion = sorted(poblacion, key=calcular_costo)  # Ordenamos por menor costo
        nueva_poblacion = poblacion[:10]  # Conservamos las mejores soluciones

        while len(nueva_poblacion) < tamano_poblacion:
            p1, p2 = random.sample(poblacion[:20], 2)  # Seleccionamos dos padres
            hijo = cruzar(p1, p2)
            if random.random() < 0.5:  # 50% de probabilidad de mutación
                hijo = mutar(hijo)

            if es_solucion_valida(hijo):
                nueva_poblacion.append(hijo)

        poblacion = nueva_poblacion

    return sorted([sol for sol in poblacion if es_solucion_valida(sol)], key=calcular_costo)[0]

# Ejecutamos el algoritmo y mostramos la mejor solución encontrada
mejor_solucion = algoritmo_genetico()
print("Mejor distribución de energía:")
print(mejor_solucion)
print("Costo total:", calcular_costo(mejor_solucion))